In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score

#import riiideducation

In [2]:
#env = riiideducation.make_env()

In [3]:
train_df = pd.read_csv('./input/riiid-test-answer-prediction/train.csv', low_memory=False, nrows=10**7,
                      dtype={
                          'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 'content_type_id': 'int8',
                              'task_container_id': 'int16', 'user_answer': 'int8', 'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32',
                             'prior_question_had_explanation': 'boolean',
                      })

# train_df = train_df.query('answered_correctly != -1').reset_index(drop=True)
# train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].astype(float) 後でやる

features_part_df と train_part_df に分ける。

In [4]:
# 900万行・・・特徴量作成
features_part_df = train_df.iloc[:int( 9 / 10 * len(train_df) )]
# 100万行・・・最新の100万件
train_part_df = train_df.iloc[int( 9 / 10 * len(train_df) ):]

In [5]:
train_part_df

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
9000000,9000000,9000000,9871729962,196122333,3992,0,242,0,1,19000.0,True
9000001,9000001,9000001,9871801320,196122333,9510,0,243,3,0,15000.0,True
9000002,9000002,9000002,9871852731,196122333,5214,0,244,3,1,13000.0,True
9000003,9000003,9000003,9872436636,196122333,5334,0,245,0,0,16000.0,True
9000004,9000004,9000004,9872617214,196122333,3722,0,246,2,1,16000.0,True
...,...,...,...,...,...,...,...,...,...,...,...
9999995,9999995,9999995,646228695,216747867,8805,0,410,1,1,23000.0,True
9999996,9999996,9999996,646321314,216747867,5708,0,411,0,0,13000.0,True
9999997,9999997,9999997,646393443,216747867,5434,0,412,0,0,66000.0,True
9999998,9999998,9999998,646445632,216747867,6429,0,413,3,1,37000.0,True


features_part_df で新しい特徴量を作成

In [6]:
# 追加するデータ1 user_answers_df
train_questions_only_df = features_part_df[features_part_df['answered_correctly'] != -1]

grouped_by_user_df = train_questions_only_df.groupby('user_id')

user_answers_df = grouped_by_user_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew']}).copy()
user_answers_df.columns = ['mean_user_accuracy', 'questions_answered', 'std_user_accuracy', 'median_user_accuracy', 'skew_user_accuracy']

In [7]:
user_answers_df

,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy
user_id,,,,,
115,0.695652,46,0.465215,1.0,-0.879359
124,0.233333,30,0.430183,0.0,1.328338
2746,0.578947,19,0.507257,1.0,-0.347892
5382,0.672000,125,0.471374,1.0,-0.741648
8623,0.642202,109,0.481566,1.0,-0.601619
...,...,...,...,...,...
196087402,0.357143,14,0.497245,0.0,0.670360
196112832,0.764706,17,0.437237,1.0,-1.372252
196116047,0.640000,25,0.489898,1.0,-0.621247


In [8]:
# 追加するデータ2 questions_df
questions_df = pd.read_csv('./input/riiid-test-answer-prediction/questions.csv')

grouped_by_content_df = train_questions_only_df.groupby('content_id')

content_answers_df = grouped_by_content_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew'] }).copy()
content_answers_df.columns = ['mean_accuracy', 'question_asked', 'std_accuracy', 'median_accuracy', 'skew_accuracy']

questions_df = questions_df.merge(content_answers_df, left_on = 'question_id', right_on = 'content_id', how = 'left')

bundle_dict = questions_df['bundle_id'].value_counts().to_dict()

# right_answers 正解数
questions_df['right_answers'] = questions_df['mean_accuracy'] * questions_df['question_asked']

questions_df['bundle_size'] = questions_df['bundle_id'].apply(lambda x: bundle_dict[x])

In [9]:
questions_df

,question_id,bundle_id,correct_answer,part,tags,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy,right_answers,bundle_size
0,0,0,0,1,51 131 162 38,0.901274,628.0,0.298532,1.0,-2.696906,566.0,1
1,1,1,1,1,131 36 81,0.889571,652.0,0.313665,1.0,-2.491635,580.0,1
2,2,2,0,1,131 101 162 92,0.558317,4064.0,0.496649,1.0,-0.234957,2269.0,1
3,3,3,0,1,131 149 162 29,0.779570,2046.0,0.414638,1.0,-1.349820,1595.0,1
4,4,4,3,1,131 5 162 38,0.627244,2841.0,0.483623,1.0,-0.526582,1782.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13518,13518,13518,3,5,14,0.782051,78.0,0.415525,1.0,-1.393293,61.0,1
13519,13519,13519,3,5,8,0.569620,79.0,0.498293,1.0,-0.286693,45.0,1
13520,13520,13520,2,5,73,0.702703,74.0,0.460188,1.0,-0.905426,52.0,1
13521,13521,13521,0,5,125,0.782051,78.0,0.415525,1.0,-1.393293,61.0,1


In [10]:
# 追加するデータ3 bundle_answers_df
grouped_by_bundle_df = questions_df.groupby('bundle_id')

bundle_answers_df = grouped_by_bundle_df.agg({'right_answers': 'sum', 'question_asked': 'sum'}).copy()
bundle_answers_df.columns = ['bundle_right_answers', 'bundle_questions_asked']

bundle_answers_df['bundle_accuracy'] = bundle_answers_df['bundle_right_answers'] / bundle_answers_df['bundle_questions_asked']

In [11]:
bundle_answers_df

,bundle_right_answers,bundle_questions_asked,bundle_accuracy
bundle_id,,,
0,566.0,628.0,0.901274
1,580.0,652.0,0.889571
2,2269.0,4064.0,0.558317
3,1595.0,2046.0,0.779570
4,1782.0,2841.0,0.627244
...,...,...,...
13518,61.0,78.0,0.782051
13519,45.0,79.0,0.569620
13520,52.0,74.0,0.702703


In [12]:
# 追加するデータ4 part_answers_df
grouped_by_part_df = questions_df.groupby('part')

part_answers_df = grouped_by_part_df.agg({'right_answers': 'sum', 'question_asked': 'sum'}).copy()

part_answers_df.columns = ['part_right_answers', 'part_questions_asked']
part_answers_df['part_accuracy'] = part_answers_df['part_right_answers'] / part_answers_df['part_questions_asked']

In [13]:
part_answers_df

,part_right_answers,part_questions_asked,part_accuracy
part,,,
1,494723.0,664481.0,0.744525
2,1189014.0,1676316.0,0.709302
3,535774.0,762586.0,0.702575
4,450117.0,711451.0,0.632675
5,2200582.0,3611081.0,0.609397
6,631060.0,944631.0,0.668049
7,300179.0,452964.0,0.662699


In [14]:
# 旧
# features = [
#     'timestamp','mean_user_accuracy', 'questions_answered','mean_accuracy',
#     'question_asked','prior_question_elapsed_time', 'prior_question_had_explanation',
#     'bundle_size', 'bundle_accuracy','part_accuracy', 'right_answers'
# ]

# 新
features = [
    'timestamp','prior_question_elapsed_time', 'prior_question_had_explanation',
    'mean_user_accuracy', 'questions_answered', 'std_user_accuracy',
    'median_user_accuracy', 'skew_user_accuracy','mean_accuracy',
    'question_asked', 'std_accuracy', 'median_accuracy', 'skew_accuracy',
    'bundle_size','bundle_accuracy', 'part_accuracy'
]

target = 'answered_correctly'

In [15]:
# 講義(-1)以外を抽出 train
train_part_df = train_part_df[train_part_df[target] != -1]

In [16]:
# user_answers_df
train_part_df = train_part_df.merge(user_answers_df, how='left', on='user_id')

# questions_df
train_part_df = train_part_df.merge(questions_df, how='left', left_on='content_id', right_on='question_id')

# bundle_answers_df
train_part_df = train_part_df.merge(bundle_answers_df, how='left', on='bundle_id')

# part_answers_df
train_part_df = train_part_df.merge(part_answers_df, how='left', on='part')

In [17]:
# ユーザーが質問に回答した後、説明と正しい回答を確認したかどうか 欠損値をFalseと置く、 astypeでデータ型の変換(キャスト)
train_part_df['prior_question_had_explanation'] = train_part_df['prior_question_had_explanation'].fillna(value=False).astype(bool)

train_part_df.fillna(value = -1, inplace = True)

In [18]:
train_part_df

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,...,median_accuracy,skew_accuracy,right_answers,bundle_size,bundle_right_answers,bundle_questions_asked,bundle_accuracy,part_right_answers,part_questions_asked,part_accuracy
0,9000000,9000000,9871729962,196122333,3992,0,242,0,1,19000.0,...,1.0,-2.172419,333.0,1,333.0,384.0,0.867188,2200582.0,3611081.0,0.609397
1,9000001,9000001,9871801320,196122333,9510,0,243,3,0,15000.0,...,1.0,-0.135984,229.0,1,229.0,429.0,0.533800,2200582.0,3611081.0,0.609397
2,9000002,9000002,9871852731,196122333,5214,0,244,3,1,13000.0,...,1.0,-0.956033,744.0,1,744.0,1040.0,0.715385,2200582.0,3611081.0,0.609397
3,9000003,9000003,9872436636,196122333,5334,0,245,0,0,16000.0,...,1.0,-0.745776,1532.0,1,1532.0,2271.0,0.674593,2200582.0,3611081.0,0.609397
4,9000004,9000004,9872617214,196122333,3722,0,246,2,1,16000.0,...,1.0,-0.976647,640.0,1,640.0,890.0,0.719101,2200582.0,3611081.0,0.609397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981089,9999995,9999995,646228695,216747867,8805,0,410,1,1,23000.0,...,1.0,-0.446302,339.0,1,339.0,557.0,0.608618,2200582.0,3611081.0,0.609397
981090,9999996,9999996,646321314,216747867,5708,0,411,0,0,13000.0,...,0.0,0.188495,2164.0,1,2164.0,4776.0,0.453099,2200582.0,3611081.0,0.609397
981091,9999997,9999997,646393443,216747867,5434,0,412,0,0,66000.0,...,1.0,-0.028372,823.0,1,823.0,1623.0,0.507086,2200582.0,3611081.0,0.609397
981092,9999998,9999998,646445632,216747867,6429,0,413,3,1,37000.0,...,1.0,-0.168422,1906.0,1,1906.0,3517.0,0.541939,2200582.0,3611081.0,0.609397


In [19]:
train_part_df.columns

Index(['Unnamed: 0', 'row_id', 'timestamp', 'user_id', 'content_id',
       'content_type_id', 'task_container_id', 'user_answer',
       'answered_correctly', 'prior_question_elapsed_time',
       'prior_question_had_explanation', 'mean_user_accuracy',
       'questions_answered', 'std_user_accuracy', 'median_user_accuracy',
       'skew_user_accuracy', 'question_id', 'bundle_id', 'correct_answer',
       'part', 'tags', 'mean_accuracy', 'question_asked', 'std_accuracy',
       'median_accuracy', 'skew_accuracy', 'right_answers', 'bundle_size',
       'bundle_right_answers', 'bundle_questions_asked', 'bundle_accuracy',
       'part_right_answers', 'part_questions_asked', 'part_accuracy'],
      dtype='object')

In [20]:
X_train = train_part_df[features]
y_train = train_part_df[target]

In [21]:
X_train

,timestamp,prior_question_elapsed_time,prior_question_had_explanation,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy,bundle_size,bundle_accuracy,part_accuracy
0,9871729962,19000.0,True,0.611722,273.0,0.488254,1.0,-0.461015,0.867188,384.0,0.339815,1.0,-2.172419,1,0.867188,0.609397
1,9871801320,15000.0,True,0.611722,273.0,0.488254,1.0,-0.461015,0.533800,429.0,0.499439,1.0,-0.135984,1,0.533800,0.609397
2,9871852731,13000.0,True,0.611722,273.0,0.488254,1.0,-0.461015,0.715385,1040.0,0.451448,1.0,-0.956033,1,0.715385,0.609397
3,9872436636,16000.0,True,0.611722,273.0,0.488254,1.0,-0.461015,0.674593,2271.0,0.468630,1.0,-0.745776,1,0.674593,0.609397
4,9872617214,16000.0,True,0.611722,273.0,0.488254,1.0,-0.461015,0.719101,890.0,0.449691,1.0,-0.976647,1,0.719101,0.609397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981089,646228695,23000.0,True,-1.000000,-1.0,-1.000000,-1.0,-1.000000,0.608618,557.0,0.488498,1.0,-0.446302,1,0.608618,0.609397
981090,646321314,13000.0,True,-1.000000,-1.0,-1.000000,-1.0,-1.000000,0.453099,4776.0,0.497848,0.0,0.188495,1,0.453099,0.609397
981091,646393443,66000.0,True,-1.000000,-1.0,-1.000000,-1.0,-1.000000,0.507086,1623.0,0.500104,1.0,-0.028372,1,0.507086,0.609397
981092,646445632,37000.0,True,-1.000000,-1.0,-1.000000,-1.0,-1.000000,0.541939,3517.0,0.498309,1.0,-0.168422,1,0.541939,0.609397


In [22]:
models = []
oof_train = np.zeros(len(X_train),) ### array([0., 0., 0., ..., 0., 0., 0.])
categorical_features = ['prior_question_had_explanation']

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

n_tr = round(981094 * 0.9)

X_tr = X_train[:n_tr]
X_val = X_train[n_tr:]

Y_tr = y_train[:n_tr]
Y_val = y_train[n_tr:]

reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_tr, Y_tr,
        eval_set=[(X_tr, Y_tr), (X_val, Y_val)],
        early_stopping_rounds=50,
        verbose=True)


oof_train = reg.predict(X_val,)

models.append(reg)

[0]	validation_0-rmse:0.46846	validation_1-rmse:0.46665
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.45212	validation_1-rmse:0.44869
[2]	validation_0-rmse:0.44389	validation_1-rmse:0.43925
[3]	validation_0-rmse:0.43960	validation_1-rmse:0.43416
[4]	validation_0-rmse:0.43745	validation_1-rmse:0.43150
[5]	validation_0-rmse:0.43638	validation_1-rmse:0.43004
[6]	validation_0-rmse:0.43581	validation_1-rmse:0.42920
[7]	validation_0-rmse:0.43542	validation_1-rmse:0.42877
[8]	validation_0-rmse:0.43520	validation_1-rmse:0.42847
[9]	validation_0-rmse:0.43501	validation_1-rmse:0.42834
[10]	validation_0-rmse:0.43486	validation_1-rmse:0.42826
[11]	validation_0-rmse:0.43477	validation_1-rmse:0.42817
[12]	validation_0-rmse:0.43469	validation_1-rmse:0.42817
[13]	validation_0-rmse:0.43457	validation_1-rmse:0.42818
[14]	validation_0-rmse:0.43449	validation_1-rmse:0.42

In [23]:
# ROC曲線のAUCスコア・・・曲線下の面積を意味するらしい。
roc_auc_score(Y_val, oof_train)

0.73189572128871

In [24]:
#@iter_test = env.iter_test()

In [25]:
#for (test_df, sample_prediction_df) in iter_test:
#    y_preds = []

#    test_df = test_df.merge(user_answers_df, how = 'left', on = 'user_id')
#    test_df = test_df.merge(questions_df, how = 'left', left_on = 'content_id', right_on = 'question_id')
#    test_df = test_df.merge(bundle_answers_df, how = 'left', on = 'bundle_id')
#    test_df = test_df.merge(part_answers_df, how = 'left', on = 'part')

#    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value = False).astype(bool)
#    test_df.fillna(value = -1, inplace = True)
#    X_test = test_df[features]

#    for model in models:
#        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
#        y_preds.append(y_pred)

#    y_preds = sum(y_preds) / len(y_preds)
#    test_df['answered_correctly'] = y_preds
#    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

## discussion

* Knowledge tracing (KT)について

Knowledge tracing (KT) refers to the problem of predicting future learner performance given their past performance in educational applications

Each student's knowledge is modeled by estimating the performance of the student on the learning activities.

KnowledgeTracing、EdNet, Artificial Intelligence in Education (AIEd)に関する記事
https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/188911

* 時制に関して

時制データには、ターゲットエンコーディングをすることが重要。(https://maxhalford.github.io/blog/pandas-tricks/#target-encoding-for-time-series)

ターゲットエンコーディングについて (https://maxhalford.github.io/blog/target-encoding/)

The one thing I do in every time series competition is target encoding. In short, the goal of target encoding is to replace a category by the average of the target values of the rows that belong to said category. Naturally, you’re not limited to using an average. You can also use Bayesian target encoding if some of your categories are rare.

Target encoding is very important for time series data. It has been used in almost every top model on Kaggle time series competitions, such as ASHRAE, Recruit Restaurants, and Wikipedia Web Traffic. When you do target encoding on temporal data, you need to be wary of not leaking current and future information into the aggregate of each row. Basically, you need to shift the target values within each group backwards. Indeed, for any given moment, we want our aggregate to pertain to past data only.

参考ディスカション (https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/189437)

* Timestamp と prior_question_elaped_time について

https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/189351
　